In [31]:
import gym, os
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np

In [32]:
import gym.spaces as gs

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.observation_space = gs.Box(shape=(4,), low=-np.inf, high=np.inf)

    def observation(self, obs):
        t_0 = (obs[0]** 2 + obs[1]**2)**(0.5)
        t_1 = (obs[2]** 2 + obs[3]**2)**(0.5)
        t_2 = obs[4]/obs[5]
        t_3 = obs[6] or obs[7]
        return [t_0, t_1, t_2, t_3]
env = gym.make("LunarLander-v2")
env = Wrapper(env)

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

In [34]:
import pennylane as qml

In [35]:
def ActorLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, y_weights, z_weights):
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            for wire, y_weight in enumerate(y_weights[layer_idx]):
                qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0)),
            qml.expval(qml.PauliZ(1)),
            qml.expval(qml.PauliZ(2)),
            qml.expval(qml.PauliZ(3))
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [36]:
def CriticLayer(n_qubits, n_layers):
    dev = qml.device("default.qubit", wires=n_qubits)

    dimensions = {
        "x_weights": (n_layers, n_qubits),
        "y_weights": (n_layers, n_qubits),
        "z_weights": (n_layers, n_qubits)
    }

    @qml.qnode(dev, interface='torch')
    def circuit(inputs, x_weights, y_weights, z_weights):
        #print(inputs)
        for layer_idx in range(n_layers):
            if (layer_idx == 0):
                for wire in range(n_qubits):
                    qml.RX(inputs[wire], wires=wire)
            for wire, x_weight in enumerate(x_weights[layer_idx]):
                qml.RX(x_weight, wires=wire)
            for wire, y_weight in enumerate(y_weights[layer_idx]):
                qml.RX(y_weight, wires=wire)
            for wire, z_weight in enumerate(z_weights[layer_idx]):
                qml.RZ(z_weight, wires=wire)
            for wire in range(n_qubits):
                qml.CNOT(wires=[wire, (wire + 1) % n_qubits])
        return [
            qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3)),
        ]

    model = qml.qnn.TorchLayer(circuit, dimensions)
    return model

In [37]:
class Actor(nn.Module):
    def __init__(self, n_layers=5):
        super(Actor, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.action_head = ActorLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.action_head(state)
        outputs = -1 + (outputs - outputs.min()) * (2) / (outputs.max() - outputs.min())
        distribution = Categorical(F.softmax(outputs, dim=-1))
        return distribution


"""class Critic(nn.Module):
    def __init__(self, n_layers=4):
        super(Critic, self).__init__()
        self.n_qubits = 4
        self.n_actions = 4
        #self.data_reupload = data_reupload
        self.critic_head = CriticLayer(n_qubits=self.n_qubits, n_layers=n_layers)

    def forward(self, state):
        state = torch.atan(state)
        outputs = self.critic_head(state)
        return outputs"""

class Critic(nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = nn.Linear(self.state_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

In [38]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns



In [39]:

def trainIters(actor, critic, n_iters):
    optimizerA = optim.Adam(actor.parameters(), lr = 0.005, amsgrad = True)
    optimizerC = optim.Adam(critic.parameters(), lr = 0.005, amsgrad = True)
    for iter in range(n_iters):
        state, _ = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()
        total_rewards = 0
        done = False

        while not done:
            #env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, term, trunc, _ = env.step(action.cpu().numpy())
            done = term or trunc

            total_rewards += reward
            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, total_rewards))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = compute_returns(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()
    torch.save(actor.state_dict(), 'actorql1.pkl')
    torch.save(critic.state_dict(), 'criticql1.pkl')
    env.close()



In [40]:

if __name__ == '__main__':
    if os.path.exists('quantum/actorql1.pkl'):
        actor = torch.load('quantum/actorql1.pkl')
        print('Actor Model loaded')
    else:
        actor = Actor().to(device)
    if os.path.exists('quantum/criticql1.pkl'):
        critic = torch.load('quantum/criticql1.pkl')
        print('Critic Model loaded')
    else:
        critic = Critic(state_size, action_size).to(device)
    trainIters(actor, critic, n_iters=1000)

Iteration: 0, Score: -611.8362329029568
Iteration: 1, Score: -501.77978185087335
Iteration: 2, Score: -722.6475716237463
Iteration: 3, Score: -603.2053967628894
Iteration: 4, Score: -379.5472822481725
Iteration: 5, Score: -335.0023167479942
Iteration: 6, Score: -440.44967279800505
Iteration: 7, Score: -357.1581018119807
Iteration: 8, Score: -381.67574840229605
Iteration: 9, Score: -468.1159660544229
Iteration: 10, Score: -316.7275142789482
Iteration: 11, Score: -455.60949824168824
Iteration: 12, Score: -510.9203031164707
Iteration: 13, Score: -377.0487063762597
Iteration: 14, Score: -339.4080289548489
Iteration: 15, Score: -334.9428008480703
Iteration: 16, Score: -696.5024181662151
Iteration: 17, Score: -282.07605228680313
Iteration: 18, Score: -366.7912417651594
Iteration: 19, Score: -170.8784058196291
Iteration: 20, Score: -298.45574791660135
Iteration: 21, Score: -363.42159390298997
Iteration: 22, Score: -408.4773136552666
Iteration: 23, Score: -535.5279964817178
Iteration: 24, Scor